# Generating surface plots

_Written by Lori Garzio, March 10, 2025_

This tutorial outlines methods for generating pcolormesh surface maps of variables grabbed from the Rutgers University Weather Research and Forecasting Model (RU-WRF 4.1) from the [RUWRF THREDDs server](http://tds.marine.rutgers.edu/thredds/cool/ruwrf/catalog.html) to compare to the analyses described in Golbazi et al 2022 [DOI 10.1088/1748-9326/ac6e49](https://iopscience.iop.org/article/10.1088/1748-9326/ac6e49/meta).

Using the [wrf_data_wrangler_grid.py](https://github.com/rucool/wind-science/blob/master/rmi-ocean-mixing/wrf_data_wrangler_grid.py) script, we grabbed model output from the summer of 2022 (June - August) for the control model version [1km_ctrl](https://tds.marine.rutgers.edu/thredds/catalog/cool/ruwrf/wrf_4_1_1km_ctrl_processed/catalog.html), a 1km model domain without simulated turbines, as well as data from the wind farm model version [1km\_wf2km\_nyb](https://tds.marine.rutgers.edu/thredds/catalog/cool/ruwrf/case_studies/wrf_4_1_1km_wf2km_upwellingctrl_processed/catalog.html), a 1km model domain with simulated 15 MW turbines spaced 2km apart and filling all wind energy lease areas in the New York Bight as of early 2023. Monthly files were generated for the following variables and heights (if applicable):

	- variables: T2, TEMP, U, V, U10, V10, UST, Q2, TKE_PBL, HFX, TSK
	- heights: surface, 120m, 160m, 200m, 300m

After grabbing all of the model output, we generated summer 2022 averages using [data_averages.py](https://github.com/rucool/wind-science/blob/master/rmi-ocean-mixing/data_averages.py). Splitting the process into these two steps was necessary because of the length of time and computer resources it takes to grab this large volume of model output. The resulting files containing the summer 2022 averages for each variable can be downloaded [here](https://marine.rutgers.edu/~lgarzio/rmi_ocean_mixing/) if you want to skip the data wrangling and averaging process.

In [ ]:
print('test')